In [22]:
import tensorflow as tf
from tensorflow.python.ops import array_ops
import numpy as np
from PIL import Image

In [23]:
image = Image.open("./image22.png").convert('RGB')
print(image.mode)

image = np.asarray(image)
print(image.shape)
image_y = np.zeros(4)
image_y[2] = 1

image = np.reshape(image, [-1, 299, 299, 3])
image_y = np.reshape(image_y, [-1, 4])

print (image.shape)
print (image_y)

RGB
(299, 299, 3)
(1, 299, 299, 3)
[[ 0.  0.  1.  0.]]


In [24]:
#tsairain_20171031 沒有dataset
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, s, padding):
    return tf.nn.conv2d(input=x, filter=W, strides=[1, s, s, 1], padding=padding)

def max_pool(x, k, s, padding):
    return tf.nn.max_pool(x, ksize= [1, k, k, 1], strides=[1, s, s, 1], padding=padding)

def avg_pool(x, k, s, padding):
    return tf.nn.avg_pool(x, ksize= [1, k, k, 1], strides=[1, s, s, 1], padding=padding)

def Stem(img): #input 299x299x3
    w1 = weight_variable([3,3,3,32])
    b1 = bias_variable([32])
    c1 = conv2d(xs, w1, 2, 'VALID') + b1  #input

    w2 = weight_variable([3,3,32,32])
    b2 = bias_variable([32])
    c2 = conv2d(c1, w2, 1, 'VALID') + b2

    w3 = weight_variable([3,3,32,64])
    b3 = bias_variable([64])
    c3 = conv2d(c2, w3, 1, 'SAME') + b3

    #concat1 = conv2 + maxpool
    w4 = weight_variable([3,3,64,96])
    b4 = bias_variable([96])
    c4 = conv2d(c3, w4, 2, 'VALID') + b4
    max1 = max_pool(c3, 3, 2, 'VALID')
    concat1 = array_ops.concat([c4, max1], 3)

    #concat2 = 2次conv2 + 4次conv2

    #concat2_left
    w5 = weight_variable([1,1,160,64])
    b5 = bias_variable([64])
    c5 = conv2d(concat1, w5, 1, 'SAME') + b5

    w6 = weight_variable([3,3,64,96])
    b6 = bias_variable([96])
    c6 = conv2d(c5, w6, 1, 'VALID') + b6

    #concat2_right
    w7 = weight_variable([1,1,160,64])
    b7 = bias_variable([64])
    c7 = conv2d(concat1, w7, 1, 'SAME') + b7

    w8 = weight_variable([7,1,64,64])
    b8 = bias_variable([64])
    c8 = conv2d(c7, w8, 1, 'SAME') + b8

    w9 = weight_variable([1,7,64,64])
    b9 = bias_variable([64])
    c9 = conv2d(c8, w9, 1, 'SAME') + b9

    w10 = weight_variable([3,3,64,96])
    b10 = bias_variable([96])
    c10 = conv2d(c9, w10, 1, 'VALID') + b10

    concat2 = array_ops.concat([c6, c10], 3)

    #concat3 = conv2 + maxpool
    w11 = weight_variable([3,3,192,192])
    b11 = bias_variable([192])
    c11 = conv2d(concat2, w11, 2, 'VALID') + b11
    max2 = max_pool(concat2, 2, 2, 'VALID')
    stem = array_ops.concat([c11, max2], 3)  #output Stem
    return stem

def InceptionA(img): #input 35x35x384
    avg = avg_pool(img, 7, 1, 'SAME')  #ksize不確定
    w1 = weight_variable([1,1,384,96])
    b1 = bias_variable([96])
    c1 = conv2d(avg, w1, 1, 'SAME') + b1
    
    w2 = weight_variable([1,1,384,96])
    b2 = bias_variable([96])
    c2 = conv2d(img, w2, 1, 'SAME') + b2
    
    w3_1 = weight_variable([1,1,384,64])
    b3_1 = bias_variable([64])
    c3_1 = conv2d(img, w3_1, 1, 'SAME') + b3_1
    w3_2 = weight_variable([3,3,64,96])
    b3_2 = bias_variable([96])
    c3_2 = conv2d(c3_1, w3_2, 1, 'SAME') + b3_2
    
    w4_1 = weight_variable([1,1,384,64])
    b4_1 = bias_variable([64])
    c4_1 = conv2d(img, w4_1, 1, 'SAME') + b4_1
    w4_2 = weight_variable([3,3,64,96])
    b4_2 = bias_variable([96])
    c4_2 = conv2d(c4_1, w4_2, 1, 'SAME') + b4_2
    w4_3 = weight_variable([3,3,96,96])
    b4_3 = bias_variable([96])
    c4_3 = conv2d(c4_2, w4_3, 1, 'SAME') + b4_3
    
    inceptionA = array_ops.concat([c1,c2,c3_2,c4_3], 3)
    return inceptionA

def ReductionA(img): #input 35x35x384
    maxpool = max_pool(img, 3, 2, 'VALID')
    
    w1 = weight_variable([3,3,384,384])
    b1 = bias_variable([384])
    c1 = conv2d(img, w1, 2, 'VALID') + b1
    
    w2_1 = weight_variable([1,1,384,192])
    b2_1 = bias_variable([192])
    c2_1 = conv2d(img, w2_1, 1, 'SAME') + b2_1
    w2_2 = weight_variable([3,3,192,224])
    b2_2 = bias_variable([224])
    c2_2 = conv2d(c2_1, w2_2, 1, 'SAME') + b2_2
    w2_3 = weight_variable([3,3,224,256])
    b2_3 = bias_variable([256])
    c2_3 = conv2d(c2_2, w2_3, 2, 'VALID') + b2_3
    
    reductionA = array_ops.concat([maxpool, c1, c2_3], 3)
    return reductionA

def InceptionB(img): #input 17x17x1024
    avg = avg_pool(img, 7, 1, 'SAME')  #ksize不確定
    
    w1 = weight_variable([1,1,1024,128])
    b1 = bias_variable([128])
    c1 = conv2d(avg, w1, 1, 'SAME') + b1
    
    w2 = weight_variable([1,1,1024,384])
    b2 = bias_variable([384])
    c2 = conv2d(img, w2, 1, 'SAME') + b2
    
    w3_1 = weight_variable([1,1,1024,192])
    b3_1 = bias_variable([192])
    c3_1 = conv2d(img, w3_1, 1, 'SAME') + b3_1       
    w3_2 = weight_variable([1,7,192,224])
    b3_2 = bias_variable([224])
    c3_2 = conv2d(c3_1, w3_2, 1, 'SAME') + b3_2       
    w3_3 = weight_variable([1,7,224,256])
    b3_3 = bias_variable([256])
    c3_3 = conv2d(c3_2, w3_3, 1, 'SAME') + b3_3
     
    w4_1 = weight_variable([1,1,1024,192])
    b4_1 = bias_variable([192])
    c4_1 = conv2d(img, w4_1, 1, 'SAME') + b4_1
    w4_2 = weight_variable([1,7,192,192])
    b4_2 = bias_variable([192])
    c4_2 = conv2d(c4_1, w4_2, 1, 'SAME') + b4_2
    w4_3 = weight_variable([7,1,192,224])
    b4_3 = bias_variable([224])
    c4_3 = conv2d(c4_2, w4_3, 1, 'SAME') + b4_3
    w4_4 = weight_variable([1,7,224,224])
    b4_4 = bias_variable([224])
    c4_4 = conv2d(c4_3, w4_4, 1, 'SAME') + b4_4
    w4_5 = weight_variable([7,1,224,256])
    b4_5 = bias_variable([256])
    c4_5 = conv2d(c4_4, w4_5, 1, 'SAME') + b4_5
    
    inceptionB = array_ops.concat([c1, c2, c3_3, c4_5], 3)
    return inceptionB

def ReductionB(img): #input 17x17x1024
    maxpool = max_pool(img, 3, 2, 'VALID')
    
    w1_1 = weight_variable([1,1,1024,192])
    b1_1 = bias_variable([192])
    c1_1 = conv2d(img, w1_1, 1, 'SAME') + b1_1
    w1_2 = weight_variable([3,3,192,192])
    b1_2 = bias_variable([192])
    c1_2 = conv2d(c1_1, w1_2, 2, 'VALID') + b1_2
    
    w2_1 = weight_variable([1,1,1024,256])
    b2_1 = bias_variable([256])
    c2_1 = conv2d(img, w2_1, 1, 'SAME') + b2_1
    w2_2 = weight_variable([1,7,256,256])
    b2_2 = bias_variable([256])
    c2_2 = conv2d(c2_1, w2_2, 1, 'SAME') + b2_2
    w2_3 = weight_variable([7,1,256,320])
    b2_3 = bias_variable([320])
    c2_3 = conv2d(c2_2, w2_3, 1, 'SAME') + b2_3
    w2_4 = weight_variable([3,3,320,320])
    b2_4 = bias_variable([320])
    c2_4 = conv2d(c2_3, w2_4, 2, 'VALID') + b2_4
    
    reductionB = array_ops.concat([maxpool, c1_2, c2_4], 3)
    return reductionB

def InceptionC(img): #input 8x8x1536
    avgpool = avg_pool(img, 8, 1, 'SAME')
    w1 = weight_variable([1,1,1536,256])
    b1 = bias_variable([256])
    c1 = conv2d(avgpool, w1, 1, 'SAME') + b1
    
    w2 = weight_variable([1,1,1536,256])
    b2 = bias_variable([256])
    c2 = conv2d(img, w2, 1, 'SAME') + b2
    
    w3 = weight_variable([1,1,1536,384])
    b3 = bias_variable([384])
    c3 = conv2d(img, w3, 1, 'SAME') + b3
    w3_1 = weight_variable([1,3,384,256])
    b3_1 = bias_variable([256])
    c3_1 = conv2d(c3, w3_1, 1, 'SAME') + b3_1
    w3_2 = weight_variable([3,1,384,256])
    b3_2 = bias_variable([256])
    c3_2 = conv2d(c3, w3_2, 1, 'SAME') + b3_2    
    
    w4_1 = weight_variable([1,1,1536,384])
    b4_1 = bias_variable([384])
    c4_1 = conv2d(img, w4_1, 1, 'SAME') + b4_1
    w4_2 = weight_variable([1,3,384,448])
    b4_2 = bias_variable([448])
    c4_2 = conv2d(c4_1, w4_2, 1, 'SAME') + b4_2
    w4_3 = weight_variable([3,1,448,512])
    b4_3 = bias_variable([512])
    c4_3 = conv2d(c4_2, w4_3, 1, 'SAME') + b4_3
    w4_31 = weight_variable([3,1,512,256])
    b4_31 = bias_variable([256])
    c4_31 = conv2d(c4_3, w4_31, 1, 'SAME') + b4_31
    w4_32 = weight_variable([1,3,512,256])
    b4_32 = bias_variable([256])
    c4_32 = conv2d(c4_3, w4_32, 1, 'SAME') + b4_32
    
    inceptionC = array_ops.concat([c1, c2, c3_1, c3_2, c4_31, c4_32], 3)
    return inceptionC
    #---------------------------------------------------------------------------    


In [26]:
xs = tf.placeholder(tf.float32, [None, 299, 299, 3]) #RGB三個通道
ys = tf.placeholder(tf.float32, [None, 4])


stem = Stem(xs)
inceptionA4 = InceptionA(InceptionA(InceptionA(InceptionA(stem))))
reductionA = ReductionA(inceptionA4)
inceptionB7 = InceptionB(InceptionB(InceptionB(InceptionB(InceptionB(InceptionB(InceptionB(reductionA)))))))
reductionB = ReductionB(inceptionB7)
inceptionC3 = InceptionC(InceptionC(InceptionC(reductionB)))
avgPool = avg_pool(inceptionC3, 8, 1, 'VALID')[:,0,0,:]
dropOut = tf.nn.dropout(avgPool, keep_prob=0.8)
#softmax
W_fc = weight_variable([1536, 4])
b_fc = bias_variable([4])
output = tf.nn.softmax(tf.matmul(dropOut, W_fc) + b_fc)

cross_entropy = tf.reduce_mean(tf.reduce_sum(tf.square(ys-output), reduction_indices=[1]))
lr = 0.01
train_step = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(ys,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [27]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
# print('c1.shape = ' + str(c1.shape))
# print('c2.shape = ' + str(c2.shape))
# print('c3.shape = ' + str(c3.shape))
# print('concat1.shape = ' + str(concat1.shape))
# print('concat2.shape = ' + str(concat2.shape))
# print('stem.shape = ' + str(stem.shape))

# print('inceptionA4.shape = ' + str(inceptionA4.shape))
# print('reductionA.shape = ' + str(reductionA.shape))
# print('inceptionB7.shape = ' + str(inceptionB7.shape))
# print('reductionB.shape = ' + str(reductionB.shape))
# print('inceptionC3.shape = ' + str(inceptionC3.shape))
# print('Average Pooling.shape = ' + str(avgPool.shape))
# print('Output.shape = ' + str(output.shape))

In [28]:
#計算準確度
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    
    print("After")
    poo=tf.shape(v_xs)
    poosee=tf.Session()
    print(poosee.run(poo))
    print("After")
    print("After")
    poo=tf.shape(y_pre)
    poosee=tf.Session()
    print(poosee.run(poo))
    print("After")
    print("After")
    poo=tf.shape(v_ys)
    poosee=tf.Session()
    print(poosee.run(poo))
    print("After")
    
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

In [31]:
# def print_filter(layer, filter_size, num):
#     i = 0
#     filter_no = 0
#     _print = ''
    
#     for i in range(filter_size):
#         _print = ''
#         for j in range(filter_size):
#             _print = _print + str(sess.run(layer[i,j,num,0])) + '\t'
#         print (_print)

In [33]:
for i in range(1000):
    sess.run(train_step, feed_dict={xs:image, ys:image_y})
#    if i == 1:
#        print_filter(W_conv1, 7, 2)
    if i % 8 == 0:
        lr = lr * 0.96
    if i % 100 ==0:
        train_accuracy = sess.run(accuracy, feed_dict={xs:image, ys:image_y})
        train_loss = sess.run(cross_entropy, feed_dict={xs:image, ys:image_y})
        print("epoch: {}, training accuracy: {:.12f}, loss: {:.6f}".format(i, train_accuracy, train_loss))
        print (sess.run(output, feed_dict={xs:image, ys:image_y}))
    

# for i in range (1000):
#     sess.run(train_step, feed_dict={xs:image, ys:out})  #目前沒有dataset

epoch: 0, training accuracy: 0.000000000000, loss: 0.000000
[[ 0.  0.  1.  0.]]
epoch: 100, training accuracy: 0.000000000000, loss: 2.000000
[[ 0.  0.  1.  0.]]
epoch: 200, training accuracy: 0.000000000000, loss: 0.000000
[[ 0.  0.  1.  0.]]
epoch: 300, training accuracy: 0.000000000000, loss: 2.000000
[[ 0.  0.  1.  0.]]
epoch: 400, training accuracy: 0.000000000000, loss: 2.000000
[[ 0.  0.  1.  0.]]
epoch: 500, training accuracy: 1.000000000000, loss: 0.000000
[[ 1.  0.  0.  0.]]
epoch: 600, training accuracy: 1.000000000000, loss: 2.000000
[[ 0.  0.  1.  0.]]
epoch: 700, training accuracy: 0.000000000000, loss: 0.000000
[[ 0.  0.  1.  0.]]
epoch: 800, training accuracy: 0.000000000000, loss: 2.000000
[[ 1.  0.  0.  0.]]
epoch: 900, training accuracy: 1.000000000000, loss: 0.000000
[[ 1.  0.  0.  0.]]
